In [ ]:
import requests, json

url = "https://apis.datos.gob.ar/georef/api/provincias?formato=geojson"
response = requests.get(url)
geojson = response.json()

# Inspeccionamos las primeras features
for f in geojson["features"][:30]:
    print(f["properties"])

In [ ]:
with open("data/argentina_provincias.geojson", "w", encoding="utf-8") as f:
    json.dump(geojson, f)


In [1]:
import pandas as pd

def load_data():
    df = pd.read_parquet("data/analytical_layer/tecnoStore_modelo_estrella.parquet", engine="fastparquet")
    return df

df = load_data()

print(df["Provincia"].unique())


['Buenos Aires' 'Río Negro' 'Chaco' 'Córdoba' 'Santa Fe' 'Salta' 'N/A'
 'Santa Cruz' 'Mendoza' 'Misiones']


In [2]:
import requests, json
import pandas as pd
import plotly.express as px

# GeoJSON oficial
url = "https://apis.datos.gob.ar/georef/api/provincias?formato=geojson"
geojson = requests.get(url).json()

# Provincias del GeoJSON
provincias_geojson = [f["properties"]["nombre"] for f in geojson["features"]]

# Cargar parquet
df = pd.read_parquet("data/analytical_layer/tecnoStore_modelo_estrella.parquet", engine="fastparquet")

# Agrupar ventas
ventas_prov = df.groupby("Provincia").agg({"monto_venta_ars_nominal":"sum"}).reset_index()

# Merge con todas las provincias
df_base = pd.DataFrame({"Provincia": provincias_geojson})
ventas_prov = df_base.merge(ventas_prov, on="Provincia", how="left").fillna(0)

# Choropleth
fig = px.choropleth(
    ventas_prov,
    geojson=geojson,
    locations="Provincia",
    featureidkey="properties.nombre",
    color="monto_venta_ars_nominal",
    title="Mapa de Ventas por Provincia",
    color_continuous_scale="Blues"
)

fig.update_geos(visible=True)  # sin fitbounds
fig.show()


In [5]:
import requests
import plotly.express as px
import pandas as pd

# --- 1. Cargar GeoJSON oficial de provincias ---
url = "https://apis.datos.gob.ar/georef/api/provincias?formato=geojson"
geojson = requests.get(url).json()

# --- 2. Crear DataFrame de prueba con todas las provincias ---
provincias = [f["properties"]["nombre"] for f in geojson["features"]]
ventas_prov = pd.DataFrame({
    "Provincia": provincias,
    "monto_venta_ars_nominal": range(1, len(provincias)+1)  # valores ficticios
})

# --- 3. Choropleth ---
fig = px.choropleth(
    ventas_prov,
    geojson=geojson,
    locations="Provincia",
    featureidkey="properties.nombre",
    color="monto_venta_ars_nominal",
    color_continuous_scale="Blues",
    title="Mapa de Ventas por Provincia"
)

# --- 4. Ajustar vista ---
fig.update_geos(fitbounds="locations", visible=False)
fig.show()



In [6]:

import plotly
print(plotly.__version__)


6.5.0


In [7]:
import requests
url = "https://apis.datos.gob.ar/georef/api/provincias?formato=geojson"
geojson = requests.get(url).json()

print(len(geojson["features"]))
print(geojson["features"][0].keys())
print(geojson["features"][0]["geometry"]["type"])


24
dict_keys(['geometry', 'properties', 'type'])
Point


In [8]:
import plotly.express as px
import pandas as pd

# Extraer coordenadas de cada provincia
provincias = []
for f in geojson["features"]:
    nombre = f["properties"]["nombre"]
    lon, lat = f["geometry"]["coordinates"]
    provincias.append({"Provincia": nombre, "lat": lat, "lon": lon})

df_geo = pd.DataFrame(provincias)

# Merge con tus ventas
ventas_prov = df.groupby("Provincia").agg({"monto_venta_ars_nominal":"sum"}).reset_index()
df_geo = df_geo.merge(ventas_prov, on="Provincia", how="left").fillna(0)

# Scatter mapbox
fig = px.scatter_mapbox(
    df_geo,
    lat="lat",
    lon="lon",
    size="monto_venta_ars_nominal",
    color="monto_venta_ars_nominal",
    hover_name="Provincia",
    mapbox_style="carto-positron",
    zoom=3,
    center={"lat": -38.4161, "lon": -63.6167},
    title="Ventas por Provincia (puntos centrales)"
)
fig.show()


C:\Users\chris\AppData\Local\Temp\ipykernel_6024\709093832.py:18: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



In [11]:
import geopandas as gpd

gdf = gpd.read_file("data/geo/provinciaPolygon.shp")   # archivo descargado del IGN
gdf.to_file("data/geo/argentina_provincias.geojson", driver="GeoJSON")


In [10]:
import geopandas as gpd
print(gpd.__version__)


1.1.1


In [ ]:
print(geojson["features"][0]["properties"])


In [ ]:
import json
import plotly.express as px
import pandas as pd

# --- 1. Cargar tu GeoJSON local con polígonos ---
with open("data/argentina_provincias.geojson", "r", encoding="utf-8") as f:
    geojson = json.load(f)

# --- 2. Inspeccionar las propiedades de la primera feature ---
print(geojson["features"][0]["properties"])

# --- 3. Crear DataFrame de prueba con todas las provincias ---
# Ajustá la clave según lo que veas en el print (ej. "nombre", "NOMBRE", etc.)
provincias = [f["properties"]["nombre"] for f in geojson["features"]]
ventas_prov = pd.DataFrame({
    "Provincia": provincias,
    "monto_venta_ars_nominal": range(1, len(provincias)+1)  # valores ficticios
})

# --- 4. Choropleth ---
fig = px.choropleth(
    ventas_prov,
    geojson=geojson,
    locations="Provincia",
    featureidkey="properties.nombre",   # ajustá según el campo que viste en el print
    color="monto_venta_ars_nominal",
    color_continuous_scale="Blues",
    title="Mapa de Ventas por Provincia"
)

# --- 5. Ajustar vista ---
fig.update_geos(fitbounds="locations", visible=False)
fig.show()


KeyError: 'NOMBRE'